In [6]:
import os
import requests
import json
import openai
import pickle as pkl
import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from time import sleep
from tqdm.auto import tqdm
import numpy as np
import ast
from sentence_transformers import SentenceTransformer

from sklearn.manifold import TSNE
import plotly.express as px


In [8]:
with open('azure-configuration.json') as inputfile:
    azureconfig = json.load(inputfile)
openai.api_key = azureconfig['key'] 
openai.api_base = azureconfig['endpoint'] 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name= azureconfig['deployment_name']

response = openai.Embedding.create(
    input="Your text string goes here",
    engine=deployment_name
)
embeddings = response['data'][0]['embedding']
print(len(embeddings))

APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
df_IAB=pd.read_csv('data/IAB_Evaluation.csv')   #Loading IAB Labels

In [ ]:
iab_labels=df_IAB['IAB_Label'].to_list()

In [ ]:
df_MAF=pd.read_csv('data/MAF_Evaluation.csv')

In [ ]:
def clean(item):
    item=item.replace("_", " ")
    item=item.lower()
    return item

In [ ]:
df_MAF['item'].apply(clean)

In [ ]:
maf=df_MAF['item'].to_list()

In [ ]:
call=iab_labels+maf

In [ ]:
print(call[0:10])

['Hair Care', 'Shaving', 'Motorcycles', "Women's Formal Wear", "Women's Clothing", 'Retail Industry', 'Comedy Events', 'Household Supplies', 'Home Security', 'Polar Travel']


In [ ]:
total_embeddings=[]

In [ ]:
def create_embeddings(item,total_embeddings):
    deployment_name= azureconfig['deployment_name']

    response = openai.Embedding.create(
        input=item,
        engine=deployment_name
    )
    embeddings = response['data'][0]['embedding']
    total_embeddings.append(embeddings)

In [ ]:
for items in tqdm(call):
    try:
        items=items.replace("&", 'and')
    except:
        pass

#only for Chat GPT version
    # if items in scores.keys():
    #     # print('here')
    #     key = list(scores[items].keys())[0]
    #     create_embeddings(key,total_embeddings)
    #     continue

    create_embeddings(items,total_embeddings)

  0%|          | 0/4745 [00:00<?, ?it/s]

  2%|▏         | 79/4745 [00:10<10:17,  7.56it/s]


KeyboardInterrupt: 

In [ ]:
len(total_embeddings)

4745

In [ ]:
iab_embeddings=total_embeddings[0:160]

In [ ]:
maf_embeddings=total_embeddings[160:]

In [ ]:
nodes=maf

In [ ]:
def calculate_precision_recall(true_labels, predictions):
    true_positives = sum(1 for pred in predictions if pred in true_labels)
    false_positives = len(predictions) - true_positives
    false_negatives = len(true_labels) - true_positives
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    return precision, recall

In [ ]:
def calculate_f1_score(precision, recall):
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [ ]:
cos_sim=cosine_similarity(np.array(iab_embeddings[90]).reshape(1,-1),np.array(maf_embeddings[15]).reshape(1,-1))
print(cos_sim)

[[0.80021651]]


In [ ]:
with open("data/problematic.pkl", 'rb') as f:
    problematic=pkl.load(f)

In [ ]:
IAB_Candidates=df_MAF['IAB_Candidates'].to_list()

In [ ]:
with open("data/child_parent_dic.pkl", 'rb') as f:
    child_parent_dic=pkl.load(f)

In [ ]:
top_n_labels_new=[]
top_n_cosines_new=[]
top_n_nodes=[]
top_n_node_ids=[]
top_n_IAB_Candidates=[]
top_n_H_scores=[]
top_n_labels=[]
top_n_cosines=[]

tsne_embeddings=[]



for i in tqdm(range(len(maf_embeddings))):
    similarities=[]
    for j in range(len(iab_embeddings)):
        cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
        similarities.append([call[160+i],call[j],cos_sim])
    
    similarities.sort(key=lambda x: x[2], reverse=True)

    
    #low initial threshold
    similarities=[x for x in similarities if x[2]>=0.765]
    new_similarities=[]

    # new_similarities=[x for x in similarities if x[2]>=0.8065]

    # #higher threshold for problematic keywords, high threshold for general keywords for accuracy
    for x in similarities:
        if len(new_similarities)>=15 and x[2]<0.82: #TOP N Thresholding
            break
        if x[2]>=0.82 and x[1] in problematic:  # Problematic Keyword Threshold
            new_similarities.append(x)
        elif x[2]>=0.80 and x[1] not in problematic:  # Non Problematic Keyword Threshold
            new_similarities.append(x)
        else:
            continue
    
    #adding atleast 1 adword
    if len(new_similarities)==0:
        while len(new_similarities)<1:
            new_similarities.append(similarities[len(new_similarities)])
        #print(new_similarities)

    #print(len(new_similarities))

    #force adding all parent adwords if child is predicted
    new_sim_adwords=[]
    for items in new_similarities:
        new_sim_adwords.append(items[1])

    count=0
    for items in new_similarities:
        if count==5:
            break
        try:
            parent_list=child_parent_dic[items[1]]
            for parent in parent_list:
                if parent not in new_sim_adwords:
                    new_sim_adwords.append(parent)
                    j=call.index(parent)
                    cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
                    if cos_sim>0.765: #Only adding parents if cosine similarity crosses a certain threshold
                        new_similarities.append([call[160+i],call[j],cos_sim])
        except Exception as e:
            #print(e)
            pass

        count+=1
    
        
    temp_iab_labels=[]
    temp_cosines=[]

    count=0
    for items in new_similarities:
        if count>=15 and items[2][0][0]<0.82:
            break
        tsne_embeddings.append(maf_embeddings[i])
        temp_iab_labels.append(items[1])
        temp_cosines.append(items[2][0])        
        top_n_labels_new.append(items[1])
        top_n_cosines_new.append(items[2][0][0])
        top_n_nodes.append(nodes[i])
        top_n_IAB_Candidates.append(IAB_Candidates[i])
        count+=1

    
    top_n_labels.append(temp_iab_labels)
    top_n_cosines.append(temp_cosines)

  7%|▋         | 336/4585 [00:29<05:01, 14.08it/s]

In [ ]:
len(top_n_labels_new) #Number of mappings

21814

In [ ]:
dic={'item':top_n_nodes, 
'predictions': top_n_labels_new, 'cosine_predicted': top_n_cosines_new}

In [ ]:
df_MAF_new=pd.DataFrame.from_dict(dic)

In [ ]:
df_MAF_new.to_csv('results/OpenAI_MAF_results_0.80_TSNE_Mapping.csv')

In [ ]:
#EVALUATION WITH H MAPPINGS BELOW

In [ ]:
MAF_predictions=df_MAF['IAB_Candidates'].to_list()

In [ ]:
candidate_count=0
hit_count=0
precision=0
recall=0
count=0
f1=0

for predictions, labels in zip(MAF_predictions,top_n_labels):
    count+=1
    proper=ast.literal_eval(predictions)
    prec, reca=calculate_precision_recall(predictions, labels)

    f=calculate_f1_score(prec, reca)
    precision+=prec
    recall+=reca
    f1+=f


    candidate_count+=len(proper)
    # for items in labels:
    #     if items in proper:
    #         hit_count+=1

    for items in proper:
        if items in labels:
            hit_count+=1

In [ ]:
hit_count/candidate_count

In [ ]:
precision/len(top_n_labels)

In [ ]:
recall/len(top_n_labels)

In [ ]:
f1/len(top_n_labels)

In [ ]:
#CALCULATING STATISTICS BELOW

In [ ]:
count=0
for items in tqdm(maf):
    count+=df_MAF_new[df_MAF_new['item']==items].shape[0]

print("Number of adwords predicted per keyword")
print(count/len(maf))

100%|██████████| 4585/4585 [00:06<00:00, 760.04it/s]

4.757688113413304


In [ ]:
#Creating Adword Statistics File
counts=[]
for items in tqdm(iab_labels):
    lol=df_MAF_new[df_MAF_new['predictions']==items].shape[0]
    counts.append(lol)

dic={'IAB_Label':iab_labels, 'Count':counts}

df_adwords=pd.DataFrame.from_dict(dic)
df_adwords.to_csv('results/Adwords_Statistics.csv',index=False)

100%|██████████| 160/160 [00:01<00:00, 95.09it/s]


In [ ]:
iab_labels[counts.index(max(counts))] #Most Used

'Shopping'

In [ ]:
iab_labels[counts.index(min(counts))] #Least Used

'Healthcare Industry'

In [ ]:
sum(counts)/160

1373.0625

In [ ]:
print(count/len(maf))

47.91712104689204


''' 


CHAT GPT REPLACEMENT CODE BELOW


'''

In [ ]:
#chat GPT Replacement
with open('chatGPT-alternate-name/alternate_names.pkl', 'rb') as f:
    alternate_dic=pkl.load(f)

In [ ]:
alternate_dic

In [ ]:
df_MAF_new.head()

In [ ]:
def create_embeddings_new(item):
    deployment_name= azureconfig['deployment_name']

    response = openai.Embedding.create(
        input=item,
        engine=deployment_name
    )
    embeddings = response['data'][0]['embedding']
    return embeddings

In [ ]:
scores={}
for items in tqdm(iab_labels):
    df_orig=df_MAF_new[df_MAF_new['predictions']==items]
    cosine_predicted=sum(df_orig['cosine_predicted'].to_list())/len(df_orig['cosine_predicted'].to_list())
    #print(cosine_predicted)
    #break
    scores[items]={'{}'.format(items):cosine_predicted}
    max_score=cosine_predicted
    for alternates in alternate_dic[items]:
        cos_sims=[]
        alternate_embedding=create_embeddings_new(alternates)
        for keywords in df_orig['item'].to_list():
            i=call.index(keywords)
            item_embedding=total_embeddings[i]
            cos_sims.append(cosine_similarity(np.array(alternate_embedding).reshape(1,-1),np.array(item_embedding).reshape(1,-1)))
        
        cosine_predicted=sum(cos_sims)/len(cos_sims)
        if cosine_predicted>max_score and len(alternates)<len(items):   #Cosine check then shorter length check
            max_score=cosine_predicted
            scores[items]={'{}'.format(alternates):cosine_predicted[0][0]}
            if items not in problematic:
                problematic.append(items)
            
    


In [ ]:
scores

In [ ]:
original_name=[]
best_name=[]
score=[]
for k,v in scores.items():
    original_name.append(k)
    for kk, vv in v.items():
        best_name.append(kk)
        score.append(vv)

In [ ]:
new_dic={'Original_name':original_name, "New_name":best_name, "Average Score":score }

In [ ]:
df_new_names=pd.DataFrame.from_dict(new_dic)

In [ ]:
df_new_names.to_csv('results/New_IAB_Labels_only_shorter.csv', index=False)